# TODO LIST

- [x] Choose Amsterdam / Rotterdam via parameter
- [x] Add Lags for remaining districts
- [x] Replace `lagged` variables with `diff`
- [x] Look-ahead values (e.g. 1 min 5 min 10 min 30 min)
  - [x] Prediction function should be tunable
- [x] Confusion Matrix per bin - find out model inaccuracies
- [x] Add `LightGBM` model
- [ ] Create Benchmarks
  - [x] Test accuracy (classification report)
  - [ ] train-time (...)
  - [ ] inference-time (...)


In [1]:
import joblib

import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    balanced_accuracy_score,
    classification_report,
    f1_score,
    make_scorer,
)
from sklearn.model_selection import (
    GridSearchCV,
    TimeSeriesSplit,
    cross_validate,
    train_test_split,
)
import geopandas as gpd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# from skopt import BayesSearchCV
from xgboost import XGBClassifier

In [43]:
df = pd.read_parquet("../data/final/points_per_district_full_o.parquet.gzip")
df.shape

(41100597, 3)

In [12]:
df = pd.read_csv("../src/data/raw/newdata.csv")
city_boundaries = gpd.read_file(f"../misc/rotterdam_.geojson")

In [13]:
def points_in_boundaries(
    chunk: pd.DataFrame,
    city_boundaries: gpd.GeoDataFrame,
    ts_col,
) -> pd.DataFrame:
    """
    Process points in boundaries.

    Args:
        chunk (pd.DataFrame): Input data.
        city_boundaries (gpd.GeoDataFrame): City boundaries.
        ts_col (str): Timestamp column name. Defaults to TS_COL.

    Returns:
        gpd.GeoDataFrame: Processed data.
    """

    chunk = chunk.drop_duplicates().reset_index(drop=True)
    chunk = gpd.GeoDataFrame(
        chunk,
        geometry=gpd.points_from_xy(chunk["longitude"], chunk["latitude"], crs=4326),
        crs=4326,
    )  # type: ignore

    df_left = pd.DataFrame(
        data=chunk.sindex.query(city_boundaries.geometry, predicate="intersects").T,
        columns=["district_id", "point_id"],
    ).reset_index(drop=True)

    df_right = (
        chunk.iloc[df_left["point_id"]][ts_col]
        .reset_index()
        .rename(columns={"index": "point_id", ts_col: "timestamp"})
    )

    merged = pd.merge(df_left, df_right, on="point_id")

    # Map district_id to district names
    district_codes = dict(city_boundaries.iloc[merged.district_id.unique()]["name"])
    merged["district_id"] = merged["district_id"].map(district_codes)

    return merged

In [24]:
df = df.drop_duplicates().reset_index(drop=True)
df = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"], crs=4326),
    crs=4326,
)

df_left = pd.DataFrame(
    data=df.sindex.query(city_boundaries.geometry, predicate="intersects").T,
    columns=["district_id", "point_id"],
).reset_index(drop=True)

df_right = (
    df.iloc[df_left["point_id"]]["timestamp"]
    .reset_index()
    .rename(columns={"index": "point_id", "timestamp": "timestamp"})
)

In [14]:
merged = points_in_boundaries(df, city_boundaries, ts_col="timestamp")

In [15]:
df.head()

,form_factor,system_id,timestamp,latitude,longitude
0,bicycle,hely,1.722364e+09,52.340670,4.873048
1,moped,felyx,1.722364e+09,51.912800,4.611037
2,cargo_bicycle,cargoroo,1.722364e+09,52.346012,4.908645
3,bicycle,hely,1.722364e+09,52.340670,4.873048
4,bicycle,hely,1.722364e+09,52.340670,4.873048


In [16]:
merged.head()

,district_id,point_id,timestamp
0,Prins Alexander,12007,1.722364e+09
1,Prins Alexander,8731,1.722364e+09
2,Prins Alexander,7936,1.722364e+09
3,Prins Alexander,5004,1.722364e+09
4,Prins Alexander,4999,1.722364e+09


In [40]:
df = (
    df.groupby(by=["district_id", "timestamp"])
    .agg({"point_id": "count"})
    .rename({"point_id": "crowd"}, axis=1)
    .sort_values(by="timestamp")
    .reset_index()
)

In [4]:
df = (
    df.pivot_table(
        index="timestamp", columns="district_id", values="crowd", aggfunc="sum"
    )
    .ffill()
    .bfill()
    .astype(np.uint16)
    .sort_values(by="timestamp")
    .reset_index()
)

df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
df.head()

district_id,timestamp,Bedrijvenpark Noord-West,Bedrijventerrein Schieveen,Botlek,Charlois,Delfshaven,Feijenoord,Hillegersberg-Schiebroek,Hoek van Holland,Hoogvliet,...,Noord,Overschie,Pernis,Prins Alexander,Rivium,Rotterdam Centrum,Spaanse Polder,Vondelingenplaat,Waalhaven,Ĳsselmonde
0,2024-06-18 06:21:58,18,8,1,327,520,496,513,15,4,...,391,37,1,238,1,732,46,1,10,225
1,2024-06-18 06:23:00,18,8,1,328,520,497,512,15,4,...,390,37,1,238,1,730,46,1,10,225
2,2024-06-18 06:24:02,18,8,1,329,519,498,512,15,4,...,391,37,1,238,1,732,46,1,10,225
3,2024-06-18 06:25:04,18,8,1,329,519,499,512,15,4,...,391,37,1,238,1,734,46,1,10,225
4,2024-06-18 06:26:07,18,8,1,329,517,499,515,15,4,...,391,37,1,238,1,733,46,1,10,225


In [5]:
print(
    f"Start Date: {pd.to_datetime(df['timestamp'].min(), unit='s')}\nEnd Date:   {pd.to_datetime(df['timestamp'].max(), unit='s')}"
)

Start Date: 2024-06-18 06:21:58
End Date:   2024-07-16 10:46:44


## Feature Extraction

1. Binning
2. Feature Extraction
   1. Time-related Features
      1. Hour
      2. Minute
      3. Day of week
   2. Lagged Features
      1. 1-10 Minutes
      2. 15 Minutes
      3. 30 Minutes
      4. 60 Minutes
   3. Rolling Mean Features
      1. Mean
      2. Std
      3. Var
      4. Kurtosis
      5. Skewness
   4. Exponential Smoothing Features
      1. 5 Minutes
      2. 10 Minutes
      3. 15 Minutes
      4. 30 Minutes
      5. 60 Minutes
3. Cyclic Encoding


##### Binning


In [18]:
target_district = "Bedrijventerrein Schieveen"
target_column = f'{target_district.replace(" ", "_")}_c_lvl'

out, bins = pd.qcut(df[target_district], q=[0, 0.3, 0.7, 1], labels=[0, 1, 2])

ValueError: Bin edges must be unique: Index([1.0, 1.0, 8.0, 8.0], dtype='float64', name='Bedrijventerrein Schieveen').
You can drop duplicate edges by setting the 'duplicates' kwarg

In [32]:
out.unique()

[1, 2, 0]
Categories (3, int64): [0 < 1 < 2]

In [17]:
def create_crowd_levels(df, target_district):
    target_column = f'{target_district.replace(" ", "_")}_c_lvl'

    mean_crowd = df[target_district].mean()
    std_crowd = df[target_district].std()

    # Define bins based on mean and standard deviation
    # bins = [
    #     float("-inf"),
    #     mean_crowd - 1.0 * (std_crowd if std_crowd != 0 else 1),
    #     mean_crowd - 0.35 * (std_crowd if std_crowd != 0 else 1),
    #     mean_crowd + 0.35 * (std_crowd if std_crowd != 0 else 1),
    #     mean_crowd + 1.0 * (std_crowd if std_crowd != 0 else 1),
    #     float("inf"),
    # ]

    bins = [
        float("-inf"),
        mean_crowd - 0.55 * (std_crowd if std_crowd != 0 else 1),
        mean_crowd + 0.55 * (std_crowd if std_crowd != 0 else 1),
        float("inf"),
    ]
    out = pd.cut(
        df[target_district],
        bins=bins,
        labels=list(range(len(bins) - 1)),
        include_lowest=True,
    )

    return out.astype(np.uint8), target_column

In [18]:
# Time-related features
time_related_features = {
    "hour": df["timestamp"].dt.hour.astype(np.uint8),
    "day_of_week": df["timestamp"].dt.day_of_week.astype(np.uint8),
    "minute": df["timestamp"].dt.minute.astype(np.uint8),
    "is_weekend": (df["timestamp"].dt.weekday >= 5).astype(np.uint8),
}

lagged_features = {}
rolling_features = {}
exp_smoothing_features = {}
windows = [5, 10, 15, 30] + [60 * i for i in range(1, 7)] + [60 * 24]
lags = list(range(1, 11)) + [15, 30] + [60 * i for i in range(1, 7)] + [60 * 24]

for district in df.columns[1:]:
    lagged_features |= {
        f"{district.replace(' ', '_')}_lag_{i}": df[district].shift(i).diff()
        for i in lags
    }

    rolling_features |= {
        f"{district.replace(' ', '_')}_rolling_{stat}_{window}": getattr(
            df[district].rolling(window=window), stat
        )()
        for window in windows
        for stat in ["mean", "std", "var", "skew", "kurt"]
    }

    exp_smoothing_features |= {
        f"{district.replace(' ', '_')}_ema_{window}": df[district]
        .ewm(span=window, adjust=True)
        .mean()
        for window in windows
    }

In [19]:
lagged_df = pd.concat(
    [
        # After Feature Extraction we can drop the original target column
        df,
        pd.DataFrame(lagged_features),
        pd.DataFrame(rolling_features),
        pd.DataFrame(exp_smoothing_features),
        pd.DataFrame(time_related_features),
    ],
    axis=1,
)

In [20]:
lagged_df.set_index("timestamp", inplace=True)
lagged_df.shape

(38924, 1810)

In [21]:
assert lagged_df.shape[1] == 21 + 21 * (19 + len(windows) * 5 + 11) + 4

In [22]:
city_boundaries = gpd.read_file("../misc/rotterdam_.geojson")

In [23]:
target_district = "Bedrijvenpark Noord-West"
out, target_column = create_crowd_levels(lagged_df, target_district)

lagged_df[target_column] = out

#### Data Splitting


In [24]:
step = 5
temp = lagged_df.copy(deep=True)

shifted_targets = temp[target_column].shift(-step)
temp[target_column] = shifted_targets

temp = temp.dropna().reset_index(drop=True)

temp[target_column] = temp[target_column].astype(np.uint8)


X, y = temp.drop(columns=target_column), temp[target_column]

In [25]:
# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=False
)

#### Pipeline Construction


In [28]:
num_features = X_train.select_dtypes(include=np.number).columns.tolist()

In [29]:
lgb = LGBMClassifier(
    boosting_type="gbdt",
    objective="multiclass",
    force_col_wise=True,
    num_class=3,
    num_leaves=3,
    max_depth=10,
    reg_lambda=0.7,
    colsample_bytree=0.5,
    learning_rate=0.01,
    reg_alpha=0.3,
    subsample=0.5,
    n_jobs=-1,
    verbosity=-1,
    random_state=42,
)

In [30]:
def build_pipeline(clf, num_features):
    preprocessor = ColumnTransformer(
        transformers=[
            ("scaler", StandardScaler(), num_features),
        ],
        remainder="passthrough",
    )

    return Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", clf),
        ]
    )


pipeline = build_pipeline(lgb, num_features)

##### Grid Search


In [31]:
tuned = False

In [32]:
ts_cv = TimeSeriesSplit(n_splits=5)
scoring = {
    "accuracy": make_scorer(balanced_accuracy_score),
    "f1_micro": make_scorer(f1_score, average="micro"),
}

In [33]:
if not tuned:
    param_grid = {
        # "classifier__n_estimators": np.arange(50, 300, 50),
        # "classifier__max_depth": np.arange(3, 10, 2),
        "classifier__estimator__learning_rate": [0.01, 0.1, 0.3],
        # "classifier__estimator__subsample": [0.5, 0.7, 1.0],
        # "classifier__estimator__colsample_bytree": [0.5, 0.8, 1.0],
        # "classifier__estimator__reg_alpha": np.linspace(0.3, 1.0, 3).round(1),
        # "classifier__reg_lambda": np.linspace(0.3, 1.0, 3).round(1)
    }
    print(f"====== Performing Grid Search ======\n")

    param_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=ts_cv,
        scoring=scoring,
        refit="f1_micro",
        n_jobs=1,
        verbose=1,
        error_score="raise",
    )
    param_search.fit(X_train, y_train)

    print("\n====== Grid Search Results ======")
    print(f"Best score: {param_search.best_score_:.3f}")
    print("Best parameters:")
    for key, value in param_search.best_params_.items():
        print(f"    - {key.split('__')[-1]}: {value}")

    print(
        "".join(
            f"{key.replace('_', ' ').title()}: {value.mean():.3f} ± {value.std():.3f}"
            for key, value in param_search.cv_results_.items()
            if key.startswith(("mean_test", "test"))
        )
    )

    best_model = param_search.best_estimator_
    best_model.fit(X_train, y_train)
else:
    print("Model is pre-tuned")
    model_path = f"models/{pipeline[1].__class__.__name__}_{step}.pkl"
    with open(model_path, "rb") as model:
        print("Loading Model...")
        best_model = joblib.load(model)

====== Performing Grid Search ======

Fitting 5 folds for each of 3 candidates, totalling 15 fits


ValueError: y contains previously unseen labels: [ 159 1014 5386]

In [ ]:
# # Bayesian Optimization
# bayes_space = {}

# bayes_search = BayesSearchCV(
#     pipeline,
#     search_spaces=bayes_space,
#     scoring="f1_micro",
#     cv=ts_cv,
#     verbose=1,
#     n_jobs=-1,
#     random_state=42,
# )

# bayes_search.fit(X_train, y_train)
# print(f"Best Params from Bayesian Optimization: {bayes_search.best_params_}")

In [ ]:
cv_result = cross_validate(
    best_model,
    X_train,
    y_train,
    cv=ts_cv,
    scoring=scoring,
    error_score="raise",
)

print("\n====== Train Set ======")
for key, value in cv_result.items():
    (
        print(
            f"CV-{key.replace('_', ' ').title()}: {value.mean():.3f} ± {value.std():.3f}"
        )
        if key.startswith(("mean_test", "test"))
        else None
    )

In [ ]:
cv_results_test = cross_validate(
    best_model,
    X_test,
    y_test,
    cv=ts_cv,
    scoring=scoring,
    error_score="raise",
)
print("\n====== Test Set ======")
for key, value in cv_results_test.items():
    (
        print(
            f"CV-{key.replace('_', ' ').title()}: {value.mean():.3f} ± {value.std():.3f}"
        )
        if key.startswith(("mean_test", "test"))
        else None
    )

In [ ]:
importances = (
    pd.DataFrame(data=[X_train.columns, best_model[1].feature_importances_])
    .T.rename(
        {
            0: "feature",
            1: "importance",
        },
        axis=1,
    )
    .sort_values(by="importance")
)

In [ ]:
importances.tail(10).plot.barh(x="feature", y="importance", figsize=(8, 6))
plt.title("Feature Importances")
plt.legend().set_visible(False)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importances")
plt.gca().spines[["right", "bottom", "top"]].set_visible(False)
plt.show()
plt.show()

In [ ]:
y_pred = best_model.predict(X_test)
_ = ConfusionMatrixDisplay.from_estimator(best_model, X_test, y_test, cmap="Blues")

In [ ]:
print(
    classification_report(
        y_test,
        y_pred,
        digits=3,
        target_names=["low", "medium", "high"],
    )
)
print("============================================")
print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_test, y_pred):.2%}")

#### Cross-Validation of the Testing Set:

- XGBClassifier
  - 5 minutes
    - Accuracy: 0.944 ± 0.020
    - F1 Micro: 0.944 ± 0.020
  - 10 minutes
    - Accuracy: 0.867 ± 0.097
    - F1 Micro: 0.867 ± 0.097
  - 30 minutes
    - Accuracy: 0.829 ± 0.119
    - F1 Micro: 0.829 ± 0.119
  - 60 minutes
    - Accuracy: 0.644 ± 0.212
    - F1 Micro: 0.644 ± 0.212
- LGBMClassifier
  - 5 minutes
    - Accuracy: 0.770 ± 0.141
    - F1 Micro: 0.916 ± 0.106
    - Balanced Accuracy Score: 95.91%
  - 10 minutes
    - Accuracy:
    - F1 Micro:
  - 30 minutes
    - Accuracy:
    - F1 Micro:
  - 60 minutes
    - Accuracy: 0.664 ± 0.134
    - F1 Micro: 0.750 ± 0.153
